In [1]:
from mip import *

In [2]:
# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [3]:
def readFile(filename):
    File = open(filename, "r")
    lines = File.readlines()
    File.close()
    
    # separa os arrays em '\n'
    lines = [lines.split() for lines in lines]

    # número de variáveis
    # número de restrições
    num_vars, num_res = int(lines[0][0]), int(lines[0][1])

    lines = lines[1:]

    # coeficientes da função objetivo
    coef_fo = lines[0]
    coef_fo = [float(i) for i in coef_fo]

    # coeficientes das restrições
    coef_res = lines[1:]
    coef_res = [[float(i) for i in coef_res[j]] for j in range(num_res)]

    m = Model(sense=MAXIMIZE, solver_name=CBC)

    # variáveis de decisão
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f'x_{str(i)}') for i in range(num_vars)]

    # função objetivo
    m.objective = xsum(coef_fo[i] * x[i] for i in range(num_vars))

    # restrições
    for i in range(num_res):
        m += xsum(coef_res[i][j] * x[j] for j in range(num_vars)) <= coef_res[i][num_vars]

    # 'silencia' o solver
    m.verbose = 0

    m.optimize()


    return m

Utilizaremos implementação em pilha para desenvolver o projeto.

In [4]:
class No:
    def __init__ (self, modelo):
        self.modelo = modelo
        self.Z = self.modelo.objective_value

        modelo.optimize()


In [5]:
def poda_integralidade (modelo):
    # Se todas as variáveis são inteiras, o nó é viável
    # logo, deve ser podado
    for v in modelo.vars:
        if v.x % 1 != 0:
            return False
    return True

# poda por limitante, analisa o valor da função objetivo
def poda_limitante (lim_inf, Z):
    # Se o valor da função objetivo for menor ou igual ao limitante inferior,
    # qualquer nó abaixo incluindo o próprio nó é descartado,
    # pois já há uma solução melhor encontrada
    if Z <= lim_inf:
        return True
    else:
        return False

# poda por inviabilidade, analisa o status do modelo
def poda_inviabilidade (modelo):
    # Inviável
    if modelo.status == OptimizationStatus.INFEASIBLE:
        return True
    # Não encontrou solução
    elif modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
        return True
    
    return False

In [6]:
def print_no(filho: Model):
    print("Status = ", filho.status)
    print(f"Solution value  = {filho.objective_value:.2f}\n")

    print("Solution:")
    for v in filho.vars:
        print(f"{v.name} = {v.x:.2f}")

In [7]:
def closest_value(array, value): 
    array = np.asarray(array)
    
    value_found = np.absolute(array - value)
    value_found = value_found.argmin()
    
    return value_found 

In [8]:
# Gera os dois nós filhos do algoritmo a partir da variável mais próxima de 0.5
def passar_no (pai):
    # escolhe a variável mais próxima de 0.5
    var_escolhida = pai.modelo.vars[closest_value([i.x for i in pai.modelo.vars], 0.5)].name

    # cria os modelos filhos
    m1 = pai.modelo.copy()
    m2 = pai.modelo.copy()

    # adiciona restrição para a variável escolhida
    m1 += m1.vars[var_escolhida] == 0
    m2 += m2.vars[var_escolhida] == 1

    # cria os nós filhos
    filho1 = No(m1)
    filho2 = No(m2)

    return filho1, filho2


In [13]:
class Branch_and_Bound:
    def __init__ (self, no_modelo):
        self.raiz = no_modelo

    def solve(self):
        lim_inf = float('-inf')
        fronteira = [self.raiz]
        solucao = "Nenhuma solução encontrada."
        
        while fronteira:
            no = fronteira[0]

            if poda_integralidade(no.modelo):
                if no.Z is not None and no.Z > lim_inf:
                    lim_inf = no.Z
                    solucao  = no.modelo
                fronteira.pop(0)
                continue
            
            if poda_inviabilidade(no.modelo):
                fronteira.pop(0)
                continue
            

            if poda_limitante(lim_inf, no.Z):
                fronteira.pop(0)
                continue

            filho1, filho2 = passar_no(no)
            fronteira.append(filho1)
            fronteira.append(filho2)

        print_no(solucao)

In [14]:
teste1 = No(readFile('example.txt'))

In [15]:
BnB = Branch_and_Bound(teste1)

In [16]:
BnB.solve()


KeyboardInterrupt: 

: 